In [1]:
import json 
from pathlib import Path

import numpy as np
import pandas as pd

In [2]:
try:
    from scipy.stats import qmc  # for Latin Hypercube
    HAS_SCIPY = True
except ImportError:
    HAS_SCIPY = False
    print("scipy not found")
    

In [48]:
PROJECT_ROOT = Path(r"C:\Users\dipes\OneDrive\Documents\Machine Learning\Python\Structural Optimization\aero_surrogate")

DV_CONFIG_PATH = PROJECT_ROOT / "config" / "dv_ranges.json"
OUTPUT_DIR = PROJECT_ROOT / "windows" / "datasets"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print("Baselines exists:", DV_CONFIG_PATH.exists())


Baselines exists: True


In [46]:
with open(DV_CONFIG_PATH, "r") as f:
    dv_config = json.load(f)

In [14]:
mins = np.array([dv_config[name]["min"] for name in dv_names], dtype=float)
maxs = np.array([dv_config[name]["max"] for name in dv_names], dtype=float)

print("Design variables:", dv_names)
print("Lower bounds:", mins)
print("Upper bounds:", maxs)

Design variables: ['dms25', 'dms50', 'dms100', 'dth25', 'dth50', 'dth100']
Lower bounds: [-0.01 -0.01 -0.01 -0.05 -0.05 -0.05]
Upper bounds: [0.01 0.01 0.01 0.05 0.05 0.05]


In [19]:
dim = len(dv_names)
N_SAMPLES = 300

In [22]:
sampler = qmc.LatinHypercube(d=dim, seed=42)
unit_samples = sampler.random(N_SAMPLES)  # shape (N_SAMPLES, dim)

In [24]:
# Scale to actual ranges: x = min + u * (max - min)
ranges = maxs - mins
samples = mins + unit_samples * ranges  # (N_SAMPLES, dim)


In [26]:
df = pd.DataFrame(samples, columns=dv_names)
df.insert(0, "case", np.arange(1, N_SAMPLES + 1))

In [28]:
print(df.head())

   case     dms25     dms50    dms100     dth25     dth50    dth100
0     1  0.006415  0.003371 -0.008791 -0.014232  0.027969  0.030008
1     2  0.007083  0.002481  0.003858 -0.047817 -0.007457  0.022691
2     3 -0.005576 -0.000522  0.004704  0.019591 -0.034518 -0.042688
3     4  0.009478 -0.000909 -0.001784 -0.024785 -0.012657  0.012036
4     5 -0.002052  0.007320  0.008102  0.041319  0.024949 -0.009561


In [39]:
out_path = OUTPUT_DIR / "designs_6dv.csv"
df.to_csv(out_path, index=False)
print("Saved DOE to:", out_path)

Saved DOE to: C:\Users\dipes\OneDrive\Documents\MachineLearning\Python\StructuralOptimization\aero_surrogate\windows\datasets\designs_6dv.csv
